<a href="https://colab.research.google.com/github/weihanchen/google-colab-python-learn/blob/main/jupyter-examples/whisper_2_channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【語音辨識 - Whisper】 雙聲道分離進行辨識

假設我們有一段雙聲道的音檔, 正常來說透過[whisper](https://github.com/openai/whisper)進行語音辨識時都是以整段音檔進行辨識,但我們若想將左右聲道分離進行辨識的話就得對音檔進行音訊處理了。

怎麼做呢？ 比較簡單的方式就是透過音訊處理工具將音檔進行左右聲道的分離，再獨立的進行辨識即可。

這次會將雙聲道音檔透過[pydub](https://github.com/jiaaro/pydub)這套音訊處理工具進行分離，再分別以`numpy.ndarray`的格式傳遞給whisper進行辨識。

## 工具安裝
- [pytube](https://pytube.io/en/latest/): 下載yt影片並轉成音檔。
- [pydub](https://github.com/jiaaro/pydub): 將雙聲道切成左、右聲道。
- [openai-whisper](https://github.com/openai/whisper): 語音辨識。

In [1]:
# 下載yt影片並轉成音檔。
!pip install pytube

# 將雙聲道切成左、右聲道。
!pip install pydub

# 安裝whisper語音辨識工具
!pip install -U openai-whisper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=4c09b834cee57a4eca371cba69db203c227812da90e1e0035796899fb49898d0
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper


### 將YT影片以Stream方式載到Memory

這邊會需要開啟google驗證網頁並將認證碼輸入才能順利下載音檔(主要也是避免無限制的下載導致負擔吧)。

1. 遵照指示打開裝置驗證網頁。
2. 輸入提示的驗證碼。
3. 登入驗證完畢後，在以下輸入框填上驗證碼即完成驗證。

In [19]:
import io
from pytube import YouTube
from pydub import AudioSegment


# yt音訊流
yt = YouTube('https://www.youtube.com/watch?v=${請自行替換音檔id}', use_oauth=True, allow_oauth_cache=True)
stream = yt.streams.filter(only_audio=True).first()

buffer = io.BytesIO()
audio_stream = stream.stream_to_buffer(buffer)
buffer.seek(0)

0

### 設計音訊解碼函式
- 預設為雙聲道layout。
- 重設取樣率,whisper預設以16000進行辨識。


In [6]:
import io
from typing import BinaryIO, Tuple, Union

import numpy as np
from pydub.utils import get_array_type


def decode_audio(
    input_file: Union[str, BinaryIO],
    sampling_rate: int = 16000,
) -> Tuple[np.ndarray, np.ndarray]:
    """Decodes the audio.

    Args:
      input_file: Path to the input file or a file-like object.
      sampling_rate: Resample the audio to this sample rate.

    Returns:
      A float32 Numpy array.

      returns a 2-tuple with the separated left and right channels.
    """
    raw_audio = AudioSegment.from_file(input_file)

    # 16-bit (2 bytes)
    raw_audio = raw_audio.set_sample_width(2)

    # 預設轉為雙聲道layout
    raw_audio = raw_audio.set_channels(2)

    # resampling
    raw_audio = raw_audio.set_frame_rate(sampling_rate)

    raw_data = raw_audio.raw_data

    dtype = get_array_type(raw_audio.sample_width * 8)
    audio = np.frombuffer(raw_data, dtype=dtype)

    # Convert s16 back to f32.
    audio = audio.astype(np.float32) / 32768.0
        
    left_channel = audio[0::2]
    right_channel = audio[1::2]
    return left_channel, right_channel

## 載入模型

有哪些模型可以使用呢? 請參考這裡: 

[https://github.com/openai/whisper#available-models-and-languages](https://github.com/openai/whisper#available-models-and-languages)

In [7]:
import whisper
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:12<00:00, 122MiB/s]


## 左聲道進行辨識


In [20]:
import numpy as np

# np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0
left, right = decode_audio(buffer)

result = model.transcribe(left, language='zh', verbose=True, initial_prompt='請給我繁體中文的語音辨識。', no_speech_threshold=1.2)

print(result)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:06.000] 就是說他們這個台中的交通怎麼會變成這麼這樣
[00:06.000 --> 00:10.000] 坐在半馬上也會被車子撞
[00:10.000 --> 00:15.000] 一次把我帶走兩個,我只有這兩個而已
[00:15.000 --> 00:20.000] 家裡多人是有重症
[00:20.000 --> 00:25.000] 只有這個女的最優秀了
[00:25.000 --> 00:28.000] 那妳是什麼時候接到消息的?
[00:28.000 --> 00:30.000] 昨天晚上
[00:30.000 --> 00:35.000] 昨天她八點還跟爸爸說我要開藥膏的藥廠
[00:35.000 --> 00:38.000] 叫我去幫她申請登記
[00:38.000 --> 00:44.000] 說她要開控室,叫我去幫她申請,我說好啊
[00:44.000 --> 00:46.000] 那妳是從台北趕下來的嗎?
[00:46.000 --> 00:48.000] 對啊
[00:48.000 --> 00:50.000] 安安哥
[00:55.000 --> 00:58.000] 市長,你說我要怎麼辦?
[01:01.000 --> 01:05.000] 今天又被妳,被妳帶走
[01:06.000 --> 01:09.000] 你說我該怎麼辦?市長
[01:19.000 --> 01:22.000] 妳要不要吃藥?
[01:37.000 --> 01:41.000] 是要帶孫子回去跟我一起住
[01:42.000 --> 01:44.000] 不要過來
[01:44.000 --> 01:46.000] 你看現在
[01:46.000 --> 01:48.000] 要好好孝順我
[01:48.000 --> 01:52.000] 只有這個女的,你看多孝順,多優秀
[01:52.000 --> 01:57.000] 她在為妳讀書,妳看,是前年那一期吧
[01:57.000 --> 02:00.000] 基本上,因為這個事發生
[02:00.000 --> 02:03.000] 所以現在為什麼會發生,怎麼也沒有消息
[02:03.000 --> 02:06.000] 那現在檢察官正在聊天當中
[02:06.000 --

## 對右聲道進行辨識

In [21]:
result = model.transcribe(right, language='zh', verbose=True, initial_prompt='請給我繁體中文的語音辨識。')

print(result)

[00:00.000 --> 00:06.000] 就是說他們這個台中的交通怎麼會變成這樣
[00:06.000 --> 00:11.000] 坐在半馬上也會被車子撞的
[00:11.000 --> 00:16.000] 一次把我帶走兩個 我只有這兩個而已
[00:16.000 --> 00:20.000] 家裡一個兒子也有重傷
[00:20.000 --> 00:25.000] 只有這個女兒最優秀而已
[00:25.000 --> 00:27.000] 那你現在什麼時候接到消息
[00:27.000 --> 00:30.000] 昨天晚上
[00:30.000 --> 00:35.000] 昨天他八點還跟爸爸說我要開藥膏的藥廠
[00:35.000 --> 00:38.000] 叫我去幫他申請登記
[00:38.000 --> 00:43.000] 說他要開控室 叫我去幫他申請不夠好
[00:44.000 --> 00:46.000] 那你是從台北上下啊
[00:46.000 --> 00:48.000] 對啊
[00:48.000 --> 00:50.000] 安安哥
[00:55.000 --> 00:58.000] 師父你說我要怎麼辦
[01:01.000 --> 01:05.000] 今天又被你帶走被帶走
[01:06.000 --> 01:09.000] 你說我開了嗎 師父
[01:19.000 --> 01:23.000] 現在都知道說還在外面
[01:33.000 --> 01:36.000] 我生來在這個大陸
[01:37.000 --> 01:41.000] 是需要帶孫子美景跟我一起住
[01:42.000 --> 01:45.000] 從那過年到現在
[01:45.000 --> 01:48.000] 要好好孝順我
[01:48.000 --> 01:50.000] 只有這個女兒你很多孝順
[01:50.000 --> 01:52.000] 多優秀
[01:52.000 --> 01:54.000] 她在外地讀書
[01:54.000 --> 01:56.000] 你解迷你解忙
[01:57.000 --> 01:59.000] 基本上因為這場發生
[01:59.000 --> 02:03.000] 現在為什麼會發生這麼嚴重的事情
[02:03